In [1]:
import json
import collections
from datetime import datetime

import plotly.graph_objects as go

# 載入免費和付費遊戲的 JSON 檔案
with open('free_games.json', 'r', encoding='utf-8') as f:
    free_games = json.load(f)

with open('paid_games.json', 'r', encoding='utf-8') as f:
    paid_games = json.load(f)

# 分析遊戲標籤，排除 'Indie' 和 'Free to Play'，並加入年份區分
def analyze_tags(games):
    tag_counts_by_year = {}
    for game in games:
        try:
            release_year = datetime.strptime(game['releaseDate'], '%b %d, %Y').year
        except ValueError:
            # 若日期格式不正確，則跳過此遊戲
            continue

        if release_year not in tag_counts_by_year:
            tag_counts_by_year[release_year] = collections.Counter()

        for tag in game['tags']:
            if tag not in ['Indie', 'Free to Play']:
                tag_counts_by_year[release_year][tag] += 1
    return tag_counts_by_year

free_game_tags_by_year = analyze_tags(free_games)
paid_game_tags_by_year = analyze_tags(paid_games)

# 計算每個標籤在每個年份的漲跌幅度
def calculate_tag_changes(tag_counts_by_year):
    tag_changes = {}
    for year, tag_counts in tag_counts_by_year.items():
        for tag, count in tag_counts.items():
            if tag not in tag_changes:
                tag_changes[tag] = {}
            tag_changes[tag][year] = count

    tag_changes_new = {}
    for tag, year_counts in tag_changes.items():
        years = sorted(year_counts.keys())
        tag_changes_new[tag] = {}
        for i in range(1, len(years)):
            prev_year = years[i - 1]
            curr_year = years[i]
            change = year_counts[curr_year] - year_counts[prev_year]
            tag_changes_new[tag][curr_year] = change
        first_year = years[0]
        if first_year not in tag_changes_new[tag]:
            tag_changes_new[tag][first_year] = year_counts[first_year]
    return tag_changes_new

free_game_tag_changes = calculate_tag_changes(free_game_tags_by_year)
paid_game_tag_changes = calculate_tag_changes(paid_game_tags_by_year)

# 計算每個標籤的總數（不考慮年份）
def calculate_total_tag_counts(games):
    tag_counts = collections.Counter()
    for game in games:
        for tag in game['tags']:
            if tag not in ['Indie', 'Free to Play']:
                tag_counts[tag] += 1
    return tag_counts

free_game_tag_totals = calculate_total_tag_counts(free_games)
paid_game_tag_totals = calculate_total_tag_counts(paid_games)

# 繪製 Plotly 圖表
def plot_tag_changes_plotly(tag_changes, title, tag_totals):
    # 按照標籤的總數排序（降序）
    sorted_tags = sorted(tag_totals, key=tag_totals.get, reverse=True)

    fig = go.Figure()
    for tag in sorted_tags:
        year_changes = tag_changes[tag]

        # 過濾年份到 2023
        years = sorted([year for year in year_changes.keys() if year <= 2023])
        changes = [year_changes[year] for year in years]

        fig.add_trace(go.Scatter(x=years, y=changes, mode='lines+markers', name=tag))

    fig.update_layout(
        title=title,
        xaxis_title='年份',
        yaxis_title='漲跌幅度',
        hovermode='closest'
    )
    fig.show()

# 繪製免費遊戲和付費遊戲的 Plotly 圖表
plot_tag_changes_plotly(free_game_tag_changes, '免費遊戲標籤漲跌幅度 (Plotly)', free_game_tag_totals)
plot_tag_changes_plotly(paid_game_tag_changes, '付費遊戲標籤漲跌幅度 (Plotly)', paid_game_tag_totals)